In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
jewellery_train = pd.read_csv('../input/datachallenge/online_jewellery_shop_train.csv')

X = jewellery_train.copy().dropna(axis=0)
y = X.pop('Revenue')

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.15)

In [ ]:
minmaxscalar = MinMaxScaler()

X_train = minmaxscalar.fit_transform(X_train)

X_valid = minmaxscalar.transform(X_valid)


In [ ]:
input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks


model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),
    layers.BatchNormalization(),
    layers.Dropout(rate=0.2),
    
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(rate=0.2),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(rate=0.1),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate=0.2),
    
    layers.Dense(128, activation='relu'),
    layers.Dropout(rate=0.2),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate=0.2),
    
    layers.Dense(128, activation='relu'),
    layers.Dropout(rate=0.1),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate=0.2),

    layers.Dense(128, activation='relu'),
    layers.Dropout(rate=0.2),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate=0.2),
    
    layers.Dense(128, activation='relu'),
    layers.Dropout(rate=0.1),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate=0.2),

    layers.Dense(128, activation='relu'),
    layers.Dropout(rate=0.2),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate=0.2),
    
    layers.Dense(128, activation='relu'),
    layers.Dropout(rate=0.1),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate=0.2),

    layers.Dense(128, activation='relu'),
    layers.Dropout(rate=0.2),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate=0.2),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(rate=0.2),
    
    layers.Dense(1, activation='sigmoid')
    ])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=64,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=0, # hide the output because we have so many epochs
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']]

# **testing**

In [ ]:
jewellery_test = pd.read_csv('../input/datachallenge/online_jewellery_shop_test_final.csv')
test_df = jewellery_test.copy().dropna(axis=0)

In [ ]:
X_test = pd.get_dummies(test_df)

In [ ]:
X_test = minmaxscalar.transform(X_test)

In [ ]:
predictions = np.concatenate(model.predict(X_test), axis=0)

In [ ]:
df_out = jewellery_test[['ID']]

In [ ]:
df_out.loc[:,'Revenue'] = 0

In [ ]:
non_null = (1-jewellery_test.isnull().any(axis=1).values).astype('bool')
df_out.loc[non_null,'Revenue'] = predictions

In [ ]:
df_out.to_csv('submission.csv', header=True, index=None)